# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'07-08-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'07-08-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-07-09 04:21:21,33.93911,67.709953,130113,5477,78521.0,46115.0,Afghanistan,334.237208,4.209418
1,NaN,NaN,NaN,Albania,2021-07-09 04:21:21,41.15330,20.168300,132565,2456,130037.0,72.0,Albania,4606.470220,1.852676
2,NaN,NaN,NaN,Algeria,2021-07-09 04:21:21,28.03390,1.659600,143652,3798,99931.0,39923.0,Algeria,327.590840,2.643889
3,NaN,NaN,NaN,Andorra,2021-07-09 04:21:21,42.50630,1.521800,14050,127,13760.0,163.0,Andorra,18184.171358,0.903915
4,NaN,NaN,NaN,Angola,2021-07-09 04:21:21,-11.20270,17.873900,39593,925,34100.0,4568.0,Angola,120.466978,2.336272


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/29/21,6/30/21,7/1/21,7/2/21,7/3/21,7/4/21,7/5/21,7/6/21,7/7/21,7/8/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,117158,118659,120216,120216,120216,120216,125937,127464,129021,130113
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,132514,132521,132523,132526,132534,132535,132537,132544,132557,132565
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,139229,139626,140075,140550,141007,141471,141966,142447,143032,143652


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/29/21,6/30/21,7/1/21,7/2/21,7/3/21,7/4/21,7/5/21,7/6/21,7/7/21,7/8/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,4794,4871,4962,4962,4962,4962,5283,5360,5415,5477
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2456,2456,2456,2456,2456,2456,2456,2456,2456,2456
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,3708,3716,3726,3735,3745,3755,3765,3775,3786,3798


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/29/21,6/30/21,7/1/21,7/2/21,7/3/21,7/4/21,7/5/21,7/6/21,7/7/21,7/8/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,70016,71012,71924,72700,73635,74480,75314,76218,77280,78521
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,130002,130009,130014,130018,130023,130027,130029,130033,130035,130037
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,96815,97089,97380,97701,98031,98387,98748,99101,99510,99931


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/29/21,6/30/21,7/1/21,7/2/21,7/3/21,7/4/21,7/5/21,7/6/21,7/7/21,7/8/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,7247,7257,7257,7262,7262,7262,7262,7262,7277,7277
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,21985,22027,22027,22043,22043,22043,22043,22043,22154,22154
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,2345,2346,2346,2347,2347,2347,2347,2347,2354,2354


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/29/21,6/30/21,7/1/21,7/2/21,7/3/21,7/4/21,7/5/21,7/6/21,7/7/21,7/8/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,113,113,113,113,113,113,113,113,113,113
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,314,315,315,315,315,315,315,315,316,316
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,60,60,60,60,60,60,60,60,60,60


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
656,1001.0,Autauga,Alabama,US,2021-07-09 04:21:21,32.539527,-86.644082,7277,113,NaN,NaN,"Autauga, Alabama, US",13025.112316,1.552838
693,1075.0,Lamar,Alabama,US,2021-07-09 04:21:21,33.779950,-88.096680,1478,38,NaN,NaN,"Lamar, Alabama, US",10706.265846,2.571042
694,1077.0,Lauderdale,Alabama,US,2021-07-09 04:21:21,34.901719,-87.656247,9674,252,NaN,NaN,"Lauderdale, Alabama, US",10432.550766,2.604920


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,33790505,606475,0.0
India,30709557,405028,29843825.0
Brazil,18962762,530179,16796161.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,33790505,606475,0.0,33184030.0,2021-07-09 04:21:21,37.950547,-91.419547
India,30709557,405028,29843825.0,460704.0,2021-07-09 04:21:21,23.088275,81.806127
Brazil,18962762,530179,16796161.0,1636422.0,2021-07-09 04:21:21,-12.669522,-48.480493
France,5861128,111473,406970.0,5342685.0,2021-07-09 04:21:21,6.430808,-34.730285
Russia,5638901,138441,5083251.0,417209.0,2021-07-09 04:21:21,54.546312,62.120860


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3829942,63815,0.0
Texas,3013216,52567,0.0
Florida,2381148,37985,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3829942,63815,0.0,3766127.0,2021-07-09 04:21:21,37.843962,-120.728594
Texas,3013216,52567,0.0,2960649.0,2021-07-09 04:21:21,31.660643,-98.653069
Florida,2381148,37985,0.0,2343163.0,2021-07-09 04:21:21,28.940755,-82.700744
New York,2118216,53721,0.0,2064495.0,2021-07-09 04:21:21,42.544151,-75.474183
Illinois,1394852,25722,0.0,1369130.0,2021-07-09 04:21:21,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1254386,24531,0.0
Arizona,Maricopa,562778,10321,0.0
Illinois,Cook,557740,10519,0.0
Florida,Miami-Dade,511394,6472,0.0
Texas,Harris,404986,6613,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1254386,24531,0.0,1229855.0,2021-07-09 04:21:21,34.308284,-118.228241,6037.0
Arizona,Maricopa,562778,10321,0.0,552457.0,2021-07-09 04:21:21,33.348359,-112.491815,4013.0
Illinois,Cook,557740,10519,0.0,547221.0,2021-07-09 04:21:21,41.841448,-87.816588,17031.0
Florida,Miami-Dade,511394,6472,0.0,504922.0,2021-07-09 04:21:21,25.611236,-80.551706,12086.0
Texas,Harris,404986,6613,0.0,398373.0,2021-07-09 04:21:21,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-06,127464,132544,142447,13991,39375,1265,4574340,225661,30861,650901,...,4975903,374019,113568,4,279813,22341,314780,6931,167132,57963
2021-07-07,129021,132557,143032,14021,39491,1265,4593763,225801,30903,651008,...,5007964,374665,114039,4,280980,23385,314869,6934,169003,60227
2021-07-08,130113,132565,143652,14050,39593,1266,4613019,225987,30951,651128,...,5040060,375298,114529,4,281907,24810,314869,6936,170456,62383


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-06,5360,2456,3775,127,920,42,96983,4530,910,10718,...,128532,5735,754,1,3222,97,3576,1363,2559,1939
2021-07-07,5415,2456,3786,127,925,42,97439,4531,910,10719,...,128565,5758,758,1,3239,102,3577,1364,2618,1973
2021-07-08,5477,2456,3798,127,925,42,97904,4536,910,10721,...,128601,5772,760,1,3253,104,3577,1365,2672,2029


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-06,76218,130033,99101,13752,34019,1222,4192546,217250,23846,638456,...,15867,358305,109349,3,261411,8077,309059,4111,147751,41624
2021-07-07,77280,130035,99510,13757,34047,1222,4206478,217303,23848,638566,...,15870,359275,109676,3,262677,8557,309251,4117,149421,42330
2021-07-08,78521,130037,99931,13760,34100,1222,4226694,217381,23850,638618,...,15888,360438,110041,3,263830,8950,309251,4119,152103,42855


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/29/21,6/30/21,7/1/21,7/2/21,7/3/21,7/4/21,7/5/21,7/6/21,7/7/21,7/8/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,7247,7257,7257,7262,7262,7262,7262,7262,7277,7277
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1472,1476,1476,1476,1476,1476,1476,1476,1478,1478
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,9640,9645,9645,9653,9653,9653,9653,9653,9674,9674
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,3136,3144,3144,3144,3144,3144,3144,3144,3146,3146
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,16319,16332,16332,16343,16343,16343,16343,16343,16390,16390


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-06,551298,71384,0,897910,352095,3824369,559921,349651,109900,49,...,868322,3006541,417259,24433,3895,681326,453665,164279,678048,62642
2021-07-07,552911,71598,0,898283,353095,3826508,560419,349743,109900,49,...,868738,3010929,417653,24440,3970,681599,454068,164346,678232,62737
2021-07-08,552911,71598,0,898908,354305,3829942,560860,349798,109931,49,...,869191,3013216,418321,24445,3970,681890,454650,164399,678459,62865


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-07-06        7262   22043    2347  2693   6988    1250   2264   14778   
2021-07-07        7277   22154    2354  2699   7013    1250   2270   14795   
2021-07-08        7277   22154    2354  2699   7013    1250   2270   14795   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-07-06         3738     1874  ...    2903    688     3273      799   
2021-07-07         3751     1885  ...    2903    693     3276      800   
2021-07-08         3751     1885  ...    2905    698     3278      800   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-07-06           4801  3804  2325          0      927    661  
2021-07-07           4805  3805  2327          0      927    663  
2021-07-08           4820  3807  2328          0      929    663  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-06,11358,377,0,18000,5926,63722,6817,8279,1695,0,...,12577,52506,2385,258,30,11434,5960,2901,8144,751
2021-07-07,11387,377,0,18004,5933,63747,6824,8278,1695,0,...,12583,52531,2387,258,32,11436,5973,2901,8154,751
2021-07-08,11387,377,0,18003,5944,63815,6827,8279,1695,0,...,12592,52567,2389,258,32,11441,5986,2904,8158,751


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-07-06         113     315      60   64    139      42     71     330   
2021-07-07         113     316      60   65    139      42     71     331   
2021-07-08         113     316      60   65    139      42     71     331   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-07-06          123       45  ...      33     13       31        7   
2021-07-07          123       45  ...      33     13       31        7   
2021-07-08          123       45  ...      33     13       31        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-07-06             40    11    13          0       26      6  
2021-07-07             40    11    13          0       26      6  
2021-07-08             40    11    13          0       26      6  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-06,0.012125,0.000053,0.003388,0.005245,0.001908,0.000791,0.004742,0.000244,0.000973,0.000129,...,0.005787,0.001964,0.004387,0.0,0.004120,0.048283,0.000671,0.000289,0.009782,0.034795
2021-07-07,0.012215,0.000098,0.004107,0.002144,0.002946,0.000000,0.004246,0.000620,0.001361,0.000164,...,0.006443,0.001727,0.004147,0.0,0.004171,0.046730,0.000283,0.000433,0.011195,0.039059
2021-07-08,0.008464,0.000060,0.004335,0.002068,0.002583,0.000791,0.004192,0.000824,0.001553,0.000184,...,0.006409,0.001690,0.004297,0.0,0.003299,0.060936,0.000000,0.000288,0.008597,0.035798


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-06,0.014575,0.0,0.002656,0.0,0.005464,0.0,0.004787,0.000663,0.0,0.000467,...,0.000288,0.004202,0.003995,0.0,0.004051,0.077778,0.00084,0.000734,0.026886,0.014652
2021-07-07,0.010261,0.0,0.002914,0.0,0.005435,0.0,0.004702,0.000221,0.0,0.000093,...,0.000257,0.004010,0.005305,0.0,0.005276,0.051546,0.00028,0.000734,0.023056,0.017535
2021-07-08,0.011450,0.0,0.003170,0.0,0.000000,0.0,0.004772,0.001104,0.0,0.000187,...,0.000280,0.002431,0.002639,0.0,0.004322,0.019608,0.00000,0.000733,0.020626,0.028383


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-06,0.012003,0.000031,0.003575,0.002259,0.008299,0.0,0.003997,0.000239,0.000126,0.000122,...,0.005705,0.003855,0.002576,0.0,0.004561,0.006856,0.000923,0.000730,0.012631,0.005265
2021-07-07,0.013934,0.000015,0.004127,0.000364,0.000823,0.0,0.003323,0.000244,0.000084,0.000172,...,0.000189,0.002707,0.002990,0.0,0.004843,0.059428,0.000621,0.001459,0.011303,0.016961
2021-07-08,0.016058,0.000015,0.004231,0.000218,0.001557,0.0,0.004806,0.000359,0.000084,0.000081,...,0.001134,0.003237,0.003328,0.0,0.004389,0.045927,0.000000,0.000486,0.017949,0.012403


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-06,0.000000,0.000000,NaN,0.001003,0.000767,0.000732,0.000388,0.000501,0.000082,0.0,...,0.001055,0.000439,0.002761,0.000573,0.000000,0.000194,0.002612,0.000792,0.000059,0.003155
2021-07-07,0.002926,0.002998,NaN,0.000415,0.002840,0.000559,0.000889,0.000263,0.000000,0.0,...,0.000479,0.001459,0.000944,0.000286,0.019255,0.000401,0.000888,0.000408,0.000271,0.001517
2021-07-08,0.000000,0.000000,NaN,0.000696,0.003427,0.000897,0.000787,0.000157,0.000282,0.0,...,0.000521,0.000760,0.001599,0.000205,0.000000,0.000427,0.001282,0.000322,0.000335,0.002040


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                  \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount Bullock   
2021-07-06      0.000000  0.000000  0.000000  0.000000  0.000000     0.0   
2021-07-07      0.002066  0.005036  0.002983  0.002228  0.003578     0.0   
2021-07-08      0.000000  0.000000  0.000000  0.000000  0.000000     0.0   

Province_State                                       ...   Wyoming            \
Admin2           Butler  Calhoun  Chambers Cherokee  ...      Park    Platte   
2021-07-06      0.00000  0.00000  0.000000  0.00000  ...  0.000000  0.005848   
2021-07-07      0.00265  0.00115  0.003478  0.00587  ...  0.000000  0.007267   
2021-07-08      0.00000  0.00000  0.000000  0.00000  ...  0.000689  0.007215   

Province_State                                                                \
Admin2          Sheridan  Sublette Sweetwater     Teton     Uinta Unassigned   
2021-07-06      0.000000  0.002509   0.002715  0.000526  0.001723        NaN   
2021-07-07      0.000917  0.001252   0.000833  0.000263  0.000860        NaN   
2021-07-08      0.000611  0.000000   0.003122  0.000526  0.000430        NaN   

Province_State                      
Admin2          Washakie    Weston  
2021-07-06      0.000000  0.000000  
2021-07-07      0.000000  0.003026  
2021-07-08      0.002157  0.000000  

[3 rows x 3336 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-06,0.000000,0.0,NaN,0.001168,0.001014,0.001053,0.000000,0.000000,0.0,NaN,...,0.000477,0.000457,0.002944,0.0,0.000000,0.000262,0.003536,0.000690,0.000000,0.005355
2021-07-07,0.002553,0.0,NaN,0.000222,0.001181,0.000392,0.001027,-0.000121,0.0,NaN,...,0.000477,0.000476,0.000839,0.0,0.066667,0.000175,0.002181,0.000000,0.001228,0.000000
2021-07-08,0.000000,0.0,NaN,-0.000056,0.001854,0.001067,0.000440,0.000121,0.0,NaN,...,0.000715,0.000685,0.000838,0.0,0.000000,0.000437,0.002176,0.001034,0.000491,0.000000


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                    \
Admin2         Autauga   Baldwin Barbour      Bibb Blount Bullock Butler   
2021-07-06         0.0  0.000000     0.0  0.000000    0.0     0.0    0.0   
2021-07-07         0.0  0.003175     0.0  0.015625    0.0     0.0    0.0   
2021-07-08         0.0  0.000000     0.0  0.000000    0.0     0.0    0.0   

Province_State                             ...  Wyoming                  \
Admin2          Calhoun Chambers Cherokee  ...     Park Platte Sheridan   
2021-07-06      0.00000      0.0      0.0  ...  0.03125    0.0      0.0   
2021-07-07      0.00303      0.0      0.0  ...  0.00000    0.0      0.0   
2021-07-08      0.00000      0.0      0.0  ...  0.00000    0.0      0.0   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-07-06          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2021-07-07          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2021-07-08          0.0        0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-06,0.018394,0.000036,0.003383,0.002642,0.001913,0.000408,0.004062,0.000297,0.001043,0.000126,...,0.005532,0.001848,0.004314,2.205815e-24,0.004112,0.049138,0.000410,0.000236,0.009978,0.029223
2021-07-07,0.015305,0.000067,0.003745,0.002393,0.002429,0.000204,0.004154,0.000459,0.001202,0.000145,...,0.005988,0.001787,0.004231,1.102907e-24,0.004141,0.047934,0.000346,0.000334,0.010586,0.034141
2021-07-08,0.011884,0.000064,0.004040,0.002231,0.002506,0.000497,0.004173,0.000641,0.001378,0.000165,...,0.006198,0.001738,0.004264,5.514537e-25,0.003720,0.054435,0.000173,0.000311,0.009592,0.034970


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-06,0.024031,2.114321e-07,0.002646,2.241053e-22,0.004200,1.245847e-16,0.004919,0.000589,1.421859e-29,0.000343,...,0.000196,0.004168,0.004147,0.0,0.004944,0.054938,0.000584,0.000554,0.024260,0.014520
2021-07-07,0.017146,1.057160e-07,0.002780,1.120526e-22,0.004817,6.229235e-17,0.004811,0.000405,7.109294e-30,0.000218,...,0.000226,0.004089,0.004726,0.0,0.005110,0.053242,0.000432,0.000644,0.023658,0.016028
2021-07-08,0.014298,5.285802e-08,0.002975,5.602632e-23,0.002409,3.114617e-17,0.004791,0.000754,3.554647e-30,0.000202,...,0.000253,0.003260,0.003682,0.0,0.004716,0.036425,0.000216,0.000689,0.022142,0.022205


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-06,0.011778,0.000028,0.003561,0.001138,0.005697,0.000013,0.004406,0.000281,0.000150,0.000140,...,0.003047,0.004173,0.002909,5.293956e-23,0.004738,0.016815,0.000608,0.001318,0.015998,0.012094
2021-07-07,0.012856,0.000022,0.003844,0.000751,0.003260,0.000006,0.003864,0.000262,0.000117,0.000156,...,0.001618,0.003440,0.002950,2.646978e-23,0.004791,0.038121,0.000615,0.001389,0.013651,0.014528
2021-07-08,0.014457,0.000019,0.004037,0.000485,0.002408,0.000003,0.004335,0.000311,0.000100,0.000119,...,0.001376,0.003339,0.003139,1.323489e-23,0.004590,0.042024,0.000307,0.000937,0.015800,0.013465


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-06,0.000028,0.000062,NaN,0.000660,0.001374,0.000454,0.000432,0.000262,0.000147,8.723971e-145,...,0.000541,0.000334,0.001450,0.000300,0.000175,0.000205,0.001363,0.000416,0.000095,0.001663
2021-07-07,0.001477,0.001530,NaN,0.000538,0.002107,0.000507,0.000661,0.000263,0.000073,4.361985e-145,...,0.000510,0.000897,0.001197,0.000293,0.009715,0.000303,0.001126,0.000412,0.000183,0.001590
2021-07-08,0.000738,0.000765,NaN,0.000617,0.002767,0.000702,0.000724,0.000210,0.000178,2.180993e-145,...,0.000516,0.000828,0.001398,0.000249,0.004858,0.000365,0.001204,0.000367,0.000259,0.001815


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-07-06      0.000033  0.000042  0.000017  0.000018  0.000021  0.000028   
2021-07-07      0.001049  0.002539  0.001500  0.001123  0.001799  0.000014   
2021-07-08      0.000525  0.001269  0.000750  0.000562  0.000900  0.000007   

Province_State                                              ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers      Cherokee  ...      Park   
2021-07-06      0.000053  0.000011  0.000022 -6.400424e-08  ... -0.000008   
2021-07-07      0.001351  0.000581  0.001750  2.934867e-03  ... -0.000004   
2021-07-08      0.000676  0.000290  0.000875  1.467433e-03  ...  0.000342   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-07-06      0.003045  0.000036  0.001254   0.001482  0.000271  0.000884   
2021-07-07      0.005156  0.000476  0.001253   0.001158  0.000267  0.000872   
2021-07-08      0.006186  0.000543  0.000626   0.002140  0.000396  0.000651   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-07-06           -1.0  0.000034  0.000172  
2021-07-07           -1.0  0.000017  0.001599  
2021-07-08           -1.0  0.001087  0.000799  

[3 rows x 3336 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-06,0.000027,0.000063,NaN,0.000709,0.000836,0.000589,0.000208,0.000004,0.000019,NaN,...,0.000256,0.000329,0.001560,0.000183,2.495869e-08,0.000286,0.001817,0.000435,0.000305,0.002715
2021-07-07,0.001290,0.000031,NaN,0.000465,0.001009,0.000491,0.000617,-0.000058,0.000009,NaN,...,0.000367,0.000402,0.001199,0.000091,3.333335e-02,0.000230,0.001999,0.000217,0.000766,0.001357
2021-07-08,0.000645,0.000016,NaN,0.000205,0.001431,0.000779,0.000529,0.000031,0.000005,NaN,...,0.000541,0.000544,0.001019,0.000046,1.666667e-02,0.000334,0.002088,0.000626,0.000629,0.000679


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State       Alabama                                        \
Admin2               Autauga   Baldwin       Barbour          Bibb   
2021-07-06      9.496620e-12  0.000025  1.034494e-06  6.897717e-21   
2021-07-07      4.748310e-12  0.001600  5.172471e-07  7.812500e-03   
2021-07-08      2.374155e-12  0.000800  2.586235e-07  3.906250e-03   

Province_State                                                                \
Admin2                Blount       Bullock        Butler   Calhoun  Chambers   
2021-07-06      8.143865e-19  2.708030e-18  6.702345e-15  0.000001 -0.000063   
2021-07-07      4.071933e-19  1.354015e-18  3.351173e-15  0.001516 -0.000032   
2021-07-08      2.035966e-19  6.770076e-19  1.675586e-15  0.000758 -0.000016   

Province_State                ...   Wyoming                          \
Admin2              Cherokee  ...      Park    Platte      Sheridan   
2021-07-06      7.527338e-26  ...  0.015885  0.000331  3.210536e-36   
2021-07-07      3.763669e-26  ...  0.007943  0.000166  1.605268e-36   
2021-07-08      1.881835e-26  ...  0.003971  0.000083  8.026341e-37   

Province_State                                                          \
Admin2              Sublette    Sweetwater         Teton         Uinta   
2021-07-06      1.410417e-56  6.113297e-09  8.278423e-10  4.517509e-21   
2021-07-07      7.052087e-57  3.056648e-09  4.139211e-10  2.258755e-21   
2021-07-08      3.526044e-57  1.528324e-09  2.069606e-10  1.129377e-21   

Province_State                                         
Admin2         Unassigned      Washakie        Weston  
2021-07-06       0.752941  8.763161e-49  7.450581e-10  
2021-07-07       0.752941  4.381581e-49  3.725290e-10  
2021-07-08       0.752941  2.190790e-49  1.862645e-10  

[3 rows x 3336 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN
